# Imports 

In [7]:
import pandas as pd
import numpy as np
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Read in data and make sure relevant columns are string/character 

- San Diego data: `naics_code` and `account_key`
- NAICS details data: `naics` (North American Industry Classification Systems)

Run code below; if pulling from github, pathname should be fine; if working elsewhere may need to edit path name at read in 

In [18]:
# uncomment if u need to find your path: os.getcwd()
SD = sd_df = pd.read_csv("/Users/sydneybrown/Documents/Github/QSS20_F24/public_data/sd_df.csv")
NA = naics_df = pd.read_csv("/Users/sydneybrown/Documents/Github/QSS20_F24/public_data/naics_df.csv")

In [20]:
cols_sd_use = ["naics_code", "account_key"]
cols_naics_use = ["naics"]

sd_df[cols_sd_use] = sd_df[cols_sd_use].astype(str)
naics_df[cols_naics_use] = naics_df[cols_naics_use].astype(str)

sd_df.dtypes
naics_df.dtypes
sd_df.head()
naics_df.head()

account_key          object
dba_name             object
council_district     object
naics_code           object
naics_description    object
naics_nchar           int64
dtype: object

naics                object
naics_description    object
dtype: object

,account_key,dba_name,council_district,naics_code,naics_description,naics_nchar
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4


,naics,naics_description
0,111140,Wheat Farming
1,111160,Rice Farming
2,111150,Corn Farming
3,111110,Soybean Farming
4,111120,Oilseed (except Soybean) Farming


## "Inner join"- retain only San Diego businesses with details on their NAICS code

- Use the `naics_code` column in the San Diego business data as the join key
- Use the `naics` column in the NAICS code details data as the join key

- Do an inner join of the San Diego data onto the NAICS code details using these join keys
- After the inner join, print some examples of San Diego businesses lost in the merge
- Use value_counts() on the `naics_nchar` column in the San Diego data to see why they might have gotten lost


In [33]:
inner = sd_df.merge(naics_df, left_on="naics_code", right_on="naics")

# see what got lost 

sd_df['naics_nchar'].value_counts()

naics_nchar
5    150
4     89
6     49
3     42
2     23
Name: count, dtype: int64

## "Left join"- retain all sd businesses even if naics code isn't in `naics_df`

- Using the same join keys as above, and treating the San Diego businesses as the left hand side data, left join the naics code details onto the San Diego businesses
- Use the `indicator` argument within merge to create an indicator, `naics_merge_status`, to help with later merge diagnostics and examine sample of ones that didn't merge
- Use the `suffixes` argument within merge to add `_sd` as the left suffix, `_census` as the right suffix
- Use `naics_merge_status` in the merged result to look at a sample of San Diego businesses that weren't matched with `naics_df`

In [68]:
left = sd_df.merge(naics_df, how='left', left_on="naics_code", right_on="naics", suffixes = ['_sd', '_census'], indicator = 'naics_merge_status')

mismatched = left[left['naics_merge_status'] == 'left_only']
mismatched

,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,naics,naics_description_census,naics_merge_status
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4,NaN,NaN,left_only
5,1979053082,GRIMM VRANJES GREER STEPHAN & BRIDGMAN LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only
6,1981000840,BENINK & SLAVENS LLP,cd_7,5411,LEGAL SERVICES,4,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...
362,2006015404,9625 TOWN CENTRE PARTNERS LP,cd_3,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3,NaN,NaN,left_only
363,2007015303,WFP-GATEWAY LP,cd_3,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3,NaN,NaN,left_only
364,2007015336,WFP-MIRAMAR LP,cd_3,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3,NaN,NaN,left_only
365,2010010820,ROSSLYN LOFTS HOUSING PARTNERS LP THE,cd_6,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3,NaN,NaN,left_only


## Use group by and agg to see if there are differences in merge rates by area

- Using the left-joined dataframe created in previous step, create a boolean indicator, `is_lost`, that equals `True` if the merge indicator is equal to "left_only" (and is otherwise false)
- Find the proportion of businesses by council district that were lost in the left join (SD businesses that failed to match to `naics_df`). To do this, group by `council_district` and use the shortcut of taking the mean of the `is_lost` indicator to find the proportions lost by `council_district`. 

In [84]:
left['is_lost'] = left['naics_merge_status'] == 'left_only'

lost_proportion = left.groupby('council_district').mean('is_lost')
lost_proportion

,naics_nchar,is_lost
council_district,,
cd_1,4.447761,0.880597
cd_2,4.434783,0.804348
cd_3,4.750000,0.812500
cd_4,3.750000,1.000000
cd_5,4.291667,0.916667
cd_6,4.508772,0.859649
cd_7,4.621622,0.918919
cd_8,4.000000,0.933333
cd_9,4.250000,1.000000


## Optional challenge exercise: add lagging 0's and see if merge rate improves

You noticed earlier that a big reason for non-matches is that the San Diego tax certificate NAICS codes were often less than six digits long, while the Census ones were always 6 digits.

You wonder if this is an issue where 0's in some of the SD's data naics codes got cut off (eg 540000 became 54), and if so whether adding these lagging zeros would improve the merge rate in a left join.

- Pad the `naics_code` column in `sd_df` with 0's to get that column up to 6-digits, using one of two approaches: 
  1. `str.pad` in pandas (https://pandas.pydata.org/docs/reference/api/pandas.Series.str.pad.html)
  2. for more of a challenge, write your own function! It should check the # of digits in the naics code string and pad it with 0's at the end up to 6 digits. To execute your function, use row-wise apply: `df.apply(lambda row: funcname(row.column), axis=1)`.
- Perform the same left join as above and see how the match rate changes
- Create an indicator variable--`is_new_match`---for new matches under the padded NAICS code; compare the `naics_description` column from San Diego versus Census in the new dataset for a sample of these new matches and comment on whether the padding seems to be correct

In [86]:
# Option 1 (use built-in function)
sd_df["new_naics_code"] = sd_df.naics_code.str.pad(width=6, side='right', fillchar='0') # Apply function
sd_df.new_naics_code.apply(lambda x: len(x)).value_counts() # Verify
# Merge
left_join_2 = pd.merge(sd_df, naics_df, left_on='new_naics_code', right_on='naics', how='left', indicator="naics_merge_status",
                    suffixes=["_sd", "_census"])
print(left_join_2.shape)
left_join_2
# Checking that proportion of is_lost decreases
left_join_2["is_lost"] = left_join_2["naics_merge_status"].apply(lambda x: x == "left_only")
left_join_2.groupby("council_district").agg({ "is_lost" : lambda x: x.mean()} )

# Option 2 (write a function challenge)
def padZeros(naics):
    num_digits = len(naics)
    if num_digits < 6:
        necessary_zeros = 6 - num_digits
        for i in range(necessary_zeros):
            naics += "0"
    return naics
sd_df["new_2_naics_code"] = sd_df.apply(lambda row: padZeros(row.naics_code), axis=1) # Apply function
left_join_3 = pd.merge(sd_df, naics_df, left_on='new_2_naics_code', right_on='naics', how='left', indicator="naics_merge_status",
                    suffixes=["_sd", "_census"])
print(left_join_3.shape)
left_join_3
# Last Step: Check the results of both functions with anti join, but there's no built-in function in pandas
anti_join = left_join_3[~(left_join_3['new_2_naics_code'].isin(left_join_2['new_naics_code']) & left_join_3['naics'].isin(left_join_2['naics']))]
print(anti_join.shape)
print(anti_join.shape[0] == 0) # Should be True


new_naics_code
6    353
Name: count, dtype: int64

(446, 10)


,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,new_naics_code,naics,naics_description_census,naics_merge_status
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,541211,Offices of Certified Public Accountants,both
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4,541100,NaN,NaN,left_only
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,541211,Offices of Certified Public Accountants,both
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4,541100,NaN,NaN,left_only
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4,541200,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...
441,2010010820,ROSSLYN LOFTS HOUSING PARTNERS LP THE,cd_6,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3,541000,NaN,NaN,left_only
442,2014004525,INSTANT SMOG CHECK,cd_6,811198,ALL OTHER AUTOMOTIVE R&M,6,811198,811198,All Other Automotive Repair and Maintenance,both
443,2014004525,INSTANT SMOG CHECK,cd_6,811198,ALL OTHER AUTOMOTIVE R&M,6,811198,811198,All Other Automotive Repair and Maintenance,both
444,2014004525,INSTANT SMOG CHECK,cd_6,811198,ALL OTHER AUTOMOTIVE R&M,6,811198,811198,All Other Automotive Repair and Maintenance,both


,is_lost
council_district,
cd_1,0.546667
cd_2,0.555556
cd_3,0.348148
cd_4,1.000000
cd_5,0.483871
cd_6,0.445946
cd_7,0.560976
cd_8,0.333333
cd_9,1.000000


(446, 11)


,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,new_naics_code,new_2_naics_code,naics,naics_description_census,naics_merge_status
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,541211,541211,Offices of Certified Public Accountants,both
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4,541100,541100,NaN,NaN,left_only
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,541211,541211,Offices of Certified Public Accountants,both
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4,541100,541100,NaN,NaN,left_only
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4,541200,541200,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...
441,2010010820,ROSSLYN LOFTS HOUSING PARTNERS LP THE,cd_6,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3,541000,541000,NaN,NaN,left_only
442,2014004525,INSTANT SMOG CHECK,cd_6,811198,ALL OTHER AUTOMOTIVE R&M,6,811198,811198,811198,All Other Automotive Repair and Maintenance,both
443,2014004525,INSTANT SMOG CHECK,cd_6,811198,ALL OTHER AUTOMOTIVE R&M,6,811198,811198,811198,All Other Automotive Repair and Maintenance,both
444,2014004525,INSTANT SMOG CHECK,cd_6,811198,ALL OTHER AUTOMOTIVE R&M,6,811198,811198,811198,All Other Automotive Repair and Maintenance,both


(0, 11)
True
